# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [25]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [26]:
# Your code here; create the connection
conn = sqlite3.Connection("data/data.sqlite")

### Q.1 Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [47]:
# Your code here
q1 = """
SELECT customerNumber
    , contactLastName
    , contactFirstName
FROM customers
WHERE (SELECT customerNumber
      FROM orders
      WHERE orderDate = '2003-01-31')
;
"""

q1_result = pd.read_sql(q1, conn)
q1_result

,customerNumber,contactLastName,contactFirstName
0,103,Schmitt,Carine
1,112,King,Jean
2,114,Ferguson,Peter
3,119,Labrune,Janine
4,121,Bergulfsen,Jonas
...,...,...,...
117,486,Salazar,Rosa
118,487,Taylor,Sue
119,489,Smith,Thomas
120,495,Franco,Valarie


### Q.2 Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [36]:
# Your code here
q2 = """
SELECT SUM(quantityOrdered) AS Total_Number_of_Orders
    , productName
FROM orderdetails
JOIN products
    ON orderdetails.productCode = products.productCode
GROUP BY productName
ORDER BY Total_Number_of_Orders
;
"""

q2_result = pd.read_sql(q2, conn)
q2_result

,Total_Number_of_Orders,productName
0,767,1957 Ford Thunderbird
1,803,1970 Chevy Chevelle SS 454
2,824,1936 Mercedes Benz 500k Roadster
3,832,1911 Ford Town Car
4,855,1999 Indy 500 Monte Carlo SS
...,...,...
104,1074,1930 Buick Marquette Phaeton
105,1076,1941 Chevrolet Special Deluxe Cabriolet
106,1085,American Airlines: MD-11S
107,1111,1937 Lincoln Berline


### Q.3 Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

**A quick note on the SQL  `SELECT DISTINCT` statement:**

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [37]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q3 = """
SELECT DISTINCT COUNT(customerNumber) AS Total_Number_of_People_Ordered
    , productName
FROM orders
JOIN orderdetails
    ON orders.orderNumber = orderdetails.orderNumber
JOIN products
    ON orderdetails.productCode = products.productCode
GROUP BY productName
ORDER BY Total_Number_of_People_Ordered
;
"""

q3_result = pd.read_sql(q3, conn)
q3_result

,Total_Number_of_People_Ordered,productName
0,24,1952 Citroen-15CV
1,24,1957 Ford Thunderbird
2,25,1911 Ford Town Car
3,25,1917 Grand Touring Sedan
4,25,1932 Alfa Romeo 8C2300 Spider Sport
...,...,...
104,28,Diamond T620 Semi-Skirted Tanker
105,28,F/A 18 Hornet 1/72
106,28,HMS Bounty
107,28,P-51-D Mustang


### Q.4 Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [45]:
# Your code here
q4 = """
SELECT DISTINCT employeeNumber
    , firstName
    , lastName
    , offices.city
    , offices.officeCode
FROM employees
JOIN offices
    ON employees.officeCode = offices.officeCode
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
    ON customers.customerNumber = orders.customerNumber
JOIN orderdetails
    ON orders.orderNumber = orderdetails.orderNumber
WHERE quantityOrdered < 20
;
"""

q4_result = pd.read_sql(q4,conn)
q4_result

,employeeNumber,firstName,lastName,city,officeCode
0,1323,George,Vanauf,NYC,3
1,1165,Leslie,Jennings,San Francisco,1
2,1621,Mami,Nishi,Tokyo,5
3,1612,Peter,Marsh,Sydney,6
4,1370,Gerard,Hernandez,Paris,4
5,1611,Andy,Fixter,Sydney,6
6,1401,Pamela,Castillo,Paris,4


### Q.5 Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [49]:
# Your code here
q5 = """
SELECT employeeNumber
    , firstName
    , lastName
    , COUNT(customerNumber) AS Customers_w_over_15k_Credit_Limit
FROM employees
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY  employeeNumber
    , firstName
    , lastName
HAVING AVG(creditLimit) >= 15000
;
"""

q5_result = pd.read_sql(q5,conn)
q5_result

,employeeNumber,firstName,lastName,Customers_w_over_15k_Credit_Limit
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!